# Pulling information from organizations in SF, processing and reuploading

In [508]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [509]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## build object to pull current cluster information from SF

In [510]:
# create DB2 to Salesforce API object
db_sf = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [511]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_df = db_sf.query_data('SELECT Estimated_cluster_size__c, Organization__c,\
Semester__c, Starting_Date__c, Ending_Date__c FROM tool_usage_cluster__c')

# display
sf_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000THPHhAAP
{"id":"7505w00000THPHhAAP","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T04:53:14.000+0000","systemModstamp":"2021-01-13T04:53:15.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000THPHhAAP","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T04:53:14.000+0000","systemModstamp":"2021-01-13T04:53:17.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3240,"retries":0,"totalProcessingTime":746}
[Success] Bulk job completed successfully.


,Ending_Date__c,Estimated_cluster_size__c,Organization__c,Semester__c,Starting_Date__c
0,2013-09-20,125.0,a0r5w00000V42cCAAR,Fall,2013-09-13
1,2011-01-30,12.0,NaN,Fall,2010-09-05
2,2018-11-30,5.0,a0r5w00000V42iyAAB,Fall,2018-11-29


In [512]:
sf_df = sf_df.dropna(subset =['Organization__c'])
display(sf_df.tail(2))
display(sf_df.shape)

,Ending_Date__c,Estimated_cluster_size__c,Organization__c,Semester__c,Starting_Date__c
3238,2020-12-02,7.0,a0r5w00000V43N4AAJ,Fall,2020-11-30
3239,2020-12-09,8.0,a0r5w00000V42krAAB,Fall,2020-12-06


(2668, 5)

In [513]:
## sort sf-df by organization
org_ind_dict = {j: [] for j in sf_df['Organization__c'].to_list()} #stores relevant indexes per org
for i,j in enumerate(sf_df['Organization__c'].to_list()):
    org_ind_dict[j].append(i)
    
print(org_ind_dict)

{'a0r5w00000V42cCAAR': [0, 3, 4, 5, 6, 7, 8, 11, 12, 14, 15, 16, 17, 19, 21, 23, 26, 29, 30, 32, 33, 34, 39, 40, 41, 44, 47, 75, 82, 83, 84, 86, 87, 89, 92, 103, 106, 128, 130, 133, 184, 185, 194, 196, 220, 247, 254, 265, 275, 334, 342, 343, 344, 345, 346, 348, 349, 350, 351, 352, 353, 355, 356, 357, 364, 374, 394, 395, 396, 439, 458, 459, 466, 481, 482, 484, 488, 491, 495, 498, 505, 556, 559, 560, 574, 575, 576, 578, 579, 589, 592, 599, 616, 627, 636, 652, 653, 655, 656, 657, 658, 660, 662, 663, 664, 667, 669, 725, 728, 789, 790, 791, 803, 811, 812, 813, 814, 815, 816, 824, 825, 826, 827, 829, 831, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 886, 887, 888, 889, 890, 891, 892, 894, 895, 896, 897, 898, 899, 900, 902, 904, 913, 914, 924, 946, 950, 951, 953, 963, 975, 979, 980, 981, 983, 985, 989, 990, 1003, 1007, 1008, 1009, 1042, 110

## Aggregating descriptive statistics

In [514]:
#########################################################################################################
## 1. Avg, Max, Median Class sizes per institution
##     i) pull and calc from the dictionary
## 2. avg and median class length
##     i) pull and calc from dictionary
## 3. users statistics
##     i) total clustered users from institution (non-unique; allows duplicates)
##     ii) weekly clustered users from institution (max, avg) *** relies on all clusters + start/stop dates
##     iii) semester clustered users from institution (max, avg) 
##     iv) unique clustered users (need to pull the enter SF user base) *** This one is the lengthy one
##     CURRENTLY - iv is omitted due to time constraints
## 4. sum of classes served (i.e., unique classes per period)
##     VERY UNIQUE: weekly number of classes after the instution began to actively use NH services (i.e. not from beginning
##     of all time)
##     i) average and max number of unique classes per week
##     ii) average and max number of classes per semester
## 5. Year-over-year comparison Individual use (weekly, monthly) 
##     i) measuring time based growth for an institution
##     ii) average yearly increase in average weekly use (average weekly use over a year - compared with next years avg)
##     iii) average yearly increase in average monthly use improvement in the number of clusters
##     
#########################################################################################################

### Point 1 - avg max median class size 

In [515]:
db_1 = DB2SalesforceAPI(sf_login_params)
db_1.object_id = 'Organization__c'
db_1.external_id = 'Id'

Obtained Salesforce access token ...... True


In [516]:
unique_orgs = list(org_ind_dict.keys())
class_size_df = pd.DataFrame(columns=['Id','Avg_Cluster_Size__c','Median_Cluster_Size__c','Max_Cluster_Size__c'])
class_size_df['Id'] = unique_orgs

In [517]:
display(class_size_df.head(2))

,Id,Avg_Cluster_Size__c,Median_Cluster_Size__c,Max_Cluster_Size__c
0,a0r5w00000V42cCAAR,NaN,NaN,NaN
1,a0r5w00000V42iyAAB,NaN,NaN,NaN


In [518]:
from copy import deepcopy
import statistics

In [519]:
cs_df_counter = 0
for i in unique_orgs:
    max_csize = 0
    med_csize = []
    avg_csize = 0
    for j in org_ind_dict[i]:
        curr_size = sf_df.iloc[j,1]
        avg_csize += curr_size/len(org_ind_dict[i])
        if curr_size > max_csize:
            max_csize = deepcopy(curr_size)
        
        med_csize.append(curr_size)
    med_csize = statistics.median(med_csize)
    class_size_df.iloc[cs_df_counter,:] = [i,avg_csize,med_csize,max_csize]
    
    cs_df_counter += 1 

In [520]:
display(class_size_df.head(5))

,Id,Avg_Cluster_Size__c,Median_Cluster_Size__c,Max_Cluster_Size__c
0,a0r5w00000V42cCAAR,38.9476,15,761
1,a0r5w00000V42iyAAB,17.4706,7,66
2,a0r5w00000V42liAAB,57.7986,28,366
3,a0r5w00000V42c0AAB,14.5714,10,36
4,a0r5w00000V42ikAAB,82.6782,16,484


In [521]:
db_1.send_data(class_size_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPIjAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPIjAAP
[Success] Closing job successful. Job ID = 7505w00000THPIjAAP


In [522]:
# check status
from pprint import pprint

In [523]:
db_1.check_bulk_status()

{'id': '7505w00000THPIjAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:53:29.000+0000',
 'systemModstamp': '2021-01-13T04:53:30.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [524]:
pprint(db_1.check_bulk_failed_results())

''


### Point 2 - avg median class length

In [525]:
db_2 = DB2SalesforceAPI(sf_login_params)
db_2.object_id = 'Organization__c'
db_2.external_id = 'Id'

Obtained Salesforce access token ...... True


In [526]:
class_length_df = pd.DataFrame(columns=['Id','Avg_Class_Length__c','Median_Class_Length__c'])
class_length_df['Id'] = unique_orgs

In [527]:
import datetime

In [528]:
cl_df_counter = 0
for i in unique_orgs: 
    ## calculate the durations per class session
    durations = []
    for j in org_ind_dict[i]:
        start_date = datetime.datetime.strptime(sf_df.iloc[j,-1],'%Y-%m-%d') 
        end_date = datetime.datetime.strptime(sf_df.iloc[j,0],'%Y-%m-%d')
        
        date_diff = end_date - start_date
        durations.append(date_diff.days) ## includes >= 30days
        
    med_dur = statistics.median(durations)
    avg_dur = statistics.mean(durations)
    
    class_length_df.iloc[cl_df_counter,:] = [i,med_dur,avg_dur]
    
    cl_df_counter += 1 

In [529]:
class_length_df

,Id,Avg_Class_Length__c,Median_Class_Length__c
0,a0r5w00000V42cCAAR,6,31.4579
1,a0r5w00000V42iyAAB,7,8.88235
2,a0r5w00000V42liAAB,9,28.705
3,a0r5w00000V42c0AAB,4,17.2857
4,a0r5w00000V42ikAAB,8,16.1724
...,...,...,...
322,a0r5w00000V42z8AAB,4,4
323,a0r5w00000V43GaAAJ,32,32
324,a0r5w00000V43HQAAZ,7,7
325,a0r5w00000V42c4AAB,5,5


In [530]:
db_2.send_data(class_length_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPIoAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPIoAAP
[Success] Closing job successful. Job ID = 7505w00000THPIoAAP


In [531]:
db_2.check_bulk_status()

{'id': '7505w00000THPIoAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:53:32.000+0000',
 'systemModstamp': '2021-01-13T04:53:33.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [532]:
pprint(db_2.check_bulk_failed_results())

''


### Point 3 and 4 - user statistics and classes served statistics

In [533]:
db_3 = DB2SalesforceAPI(sf_login_params)
db_3.object_id = 'Organization__c'
db_3.external_id = 'Id'

Obtained Salesforce access token ...... True


#### total clustered users

In [534]:
tcu_df = pd.DataFrame(columns=['Id','Total_Clustered_Users__c'])
tcu_df['Id'] = unique_orgs

In [535]:
tcu_counter = 0
for i in unique_orgs:
    total_clustered_users = 0
    for j in org_ind_dict[i]:
        total_clustered_users += sf_df.iloc[j,1]
        
    tcu_df.iloc[tcu_counter,1] = deepcopy(total_clustered_users)
    
    tcu_counter += 1

In [536]:
display(tcu_df.head(5))

,Id,Total_Clustered_Users__c
0,a0r5w00000V42cCAAR,17098
1,a0r5w00000V42iyAAB,297
2,a0r5w00000V42liAAB,8034
3,a0r5w00000V42c0AAB,102
4,a0r5w00000V42ikAAB,7193


In [537]:
db_3.send_data(tcu_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPItAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPItAAP
[Success] Closing job successful. Job ID = 7505w00000THPItAAP


In [538]:
db_3.check_bulk_status()

{'id': '7505w00000THPItAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:53:35.000+0000',
 'systemModstamp': '2021-01-13T04:53:36.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [539]:
pprint(db_3.check_bulk_failed_results())

''


#### weekly clustered users and weekly classes

In [540]:
## first, need to find earliest start and latest stop per institution
estart = [] 
lstop = [] 

In [541]:
## record estart and lstop
for i in unique_orgs:
    estart_temp = datetime.datetime.strptime('2022-01-01','%Y-%m-%d')
    lstop_temp = datetime.datetime.strptime('2000-01-01','%Y-%m-%d')
    
    for j in org_ind_dict[i]: 
        start_date = datetime.datetime.strptime(sf_df.iloc[j,-1],'%Y-%m-%d') 
        end_date = datetime.datetime.strptime(sf_df.iloc[j,0],'%Y-%m-%d')
        
        if start_date <= estart_temp:
            estart_temp = deepcopy(start_date)
        
        if end_date >= lstop_temp:
            lstop_temp = deepcopy(end_date)
    
    estart.append(estart_temp) #indexed according to unique orgs
    lstop.append(lstop_temp)

In [542]:
print(len(lstop),len(estart))
print(len(unique_orgs))

327 327
327


In [543]:
## build holder variables [both are orgnization IDs]
weekly_classes_df = pd.DataFrame(columns=['Id','Avg_Week_Classes__c','Max_Week_Classes__c'])
weekly_users_df = pd.DataFrame(columns=['Id','Avg_Week_Users__c','Max_Week_Users__c'])

weekly_classes_df['Id'] = unique_orgs
weekly_users_df['Id'] = unique_orgs

In [544]:
### The sequential time based classroom calculation needs a discussion on the definition of a week
### currently - if a class ends exactly when another week begins it is counted as belonging to both weeks (e.g.)

In [545]:
## begin sequential test
# for each org and while time permits find the number of classes per week + number of people in said classes
for i in unique_orgs:
    # build list of times
    week_timing_start = estart[unique_orgs.index(i)]
    week_timing_end = week_timing_start + datetime.timedelta(days=6) #not days=7 because of arithmetic
    
    weekly_classes = []
    weekly_users = []
    
    init_week = True
    
    ## develop week_timing_end threshold
    total_week_timing = lstop[unique_orgs.index(i)] - estart[unique_orgs.index(i)]
    if total_week_timing.days % 6 == 0: # perfectly divisible
        week_end_thresh = lstop[unique_orgs.index(i)]
    else:
        ## account for undershooting (we need to add days)
        if (lstop[unique_orgs.index(i)] + datetime.timedelta(days= total_week_timing.days%6) \
            - estart[unique_orgs.index(i)]).days%6 == 0:
            week_end_thresh = lstop[unique_orgs.index(i)] + datetime.timedelta(days= total_week_timing.days%6)
        else: ## accounting for overshooting
            week_end_thresh = lstop[unique_orgs.index(i)] + datetime.timedelta(days= 7 - (total_week_timing.days%6))
    
    #print(week_end_thresh)
    
    while init_week == True or week_timing_end <= week_end_thresh:
        init_week = False
        
        this_week_classes = 0
        this_week_users = 0
        
        for j in org_ind_dict[i]:
            #check if start and end dates encompass current week
            start_date = datetime.datetime.strptime(sf_df.iloc[j,-1],'%Y-%m-%d') 
            end_date = datetime.datetime.strptime(sf_df.iloc[j,0],'%Y-%m-%d')
            
            #print(start_date,end_date)
            #print(week_timing_start,week_timing_end)
            #print(start_date <= week_timing_end)
            
            if end_date < week_timing_start or start_date > week_timing_end: #case 1 - ( ) [ ]; case 2- [ ] ( )
                this_week_classes += 0
                this_week_users += 0
            else:
                #case 3 - [ ( ) ], 4 -( [ ] ), 5 - ( [ ) ], 6 - [ ( ] )
                this_week_classes += 1
                this_week_users += sf_df.iloc[j,1]
            
            #print(this_week_classes)
            #print(this_week_users)
            
        weekly_classes.append(this_week_classes)
        weekly_users.append(this_week_users)
        
        week_timing_start += datetime.timedelta(days=6) #see title for why 6 and not 7
        week_timing_end += datetime.timedelta(days=6)
        
        #print(lstop[unique_orgs.index(i)])
        #print(week_timing_end)
        
    weekly_classes_df.iloc[unique_orgs.index(i),1] = statistics.mean(weekly_classes)
    weekly_classes_df.iloc[unique_orgs.index(i),2] = max(weekly_classes)
    
    weekly_users_df.iloc[unique_orgs.index(i),1] = statistics.mean(weekly_users)
    weekly_users_df.iloc[unique_orgs.index(i),2] = max(weekly_users)

In [546]:
display(weekly_classes_df.head(5))
display(weekly_classes_df.tail(5))

,Id,Avg_Week_Classes__c,Max_Week_Classes__c
0,a0r5w00000V42cCAAR,2.26889,8
1,a0r5w00000V42iyAAB,0.142857,3
2,a0r5w00000V42liAAB,0.870488,5
3,a0r5w00000V42c0AAB,0.0318772,1
4,a0r5w00000V42ikAAB,0.334334,4


,Id,Avg_Week_Classes__c,Max_Week_Classes__c
322,a0r5w00000V42z8AAB,1,1
323,a0r5w00000V43GaAAJ,1,1
324,a0r5w00000V43HQAAZ,1,1
325,a0r5w00000V42c4AAB,1,1
326,a0r5w00000V43DFAAZ,1,1


In [547]:
display(weekly_users_df.head(5))
display(weekly_users_df.tail(5))

,Id,Avg_Week_Users__c,Max_Week_Users__c
0,a0r5w00000V42cCAAR,218.154,1132
1,a0r5w00000V42iyAAB,3.16558,66
2,a0r5w00000V42liAAB,98.1083,517
3,a0r5w00000V42c0AAB,0.400236,36
4,a0r5w00000V42ikAAB,44.2603,1046


,Id,Avg_Week_Users__c,Max_Week_Users__c
322,a0r5w00000V42z8AAB,14,14
323,a0r5w00000V43GaAAJ,13,13
324,a0r5w00000V43HQAAZ,23,23
325,a0r5w00000V42c4AAB,11,11
326,a0r5w00000V43DFAAZ,15,15


In [548]:
db_4 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [549]:
db_4.external_id = 'Id'
db_4.object_id = 'Organization__c'

In [550]:
db_4.send_data(weekly_classes_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPJcAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPJcAAP
[Success] Closing job successful. Job ID = 7505w00000THPJcAAP


In [551]:
db_4.check_bulk_status()

{'id': '7505w00000THPJcAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:56:23.000+0000',
 'systemModstamp': '2021-01-13T04:56:26.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [552]:
pprint(db_4.check_bulk_failed_results())

''


In [553]:
db_5 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [554]:
db_5.external_id = 'Id'
db_5.object_id = 'Organization__c'

In [555]:
db_5.send_data(weekly_users_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPJhAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPJhAAP
[Success] Closing job successful. Job ID = 7505w00000THPJhAAP


In [556]:
db_5.check_bulk_status()

{'id': '7505w00000THPJhAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:56:27.000+0000',
 'systemModstamp': '2021-01-13T04:56:28.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [557]:
pprint(db_5.check_bulk_failed_results())

''


#### semester clustered users and classes (avg and max)

In [558]:
## build semester timing table
# spring > summer > fall sequence
sem_dict_timing = {'Spring':0,'Summer':1,'Fall':2}

sstart = []
sstop = []
eff_year = ['init']
end_eff_year = ['init']

In [559]:
sf_df.head(5)

,Ending_Date__c,Estimated_cluster_size__c,Organization__c,Semester__c,Starting_Date__c
0,2013-09-20,125.0,a0r5w00000V42cCAAR,Fall,2013-09-13
2,2018-11-30,5.0,a0r5w00000V42iyAAB,Fall,2018-11-29
3,2019-11-28,44.0,a0r5w00000V42iyAAB,Fall,2019-11-04
4,2017-12-16,66.0,a0r5w00000V42cCAAR,Fall,2017-10-03
5,2018-01-21,18.0,a0r5w00000V42cCAAR,Spring,2018-01-15


In [560]:
## record estart and lstop
for i in unique_orgs:
    sstart_temp = 2022
    sstop_temp = 2000
    
    earliest = 5 #default NaN
    latest = 5 #default NaN
    
    for j in org_ind_dict[i]: 
        sem = sf_df.iloc[j,-2]
        
        syear = datetime.datetime.strptime(sf_df.iloc[j,-1],'%Y-%m-%d') 
        eyear = datetime.datetime.strptime(sf_df.iloc[j,0],'%Y-%m-%d') 
        
        if syear.year < sstart_temp:
            sstart_temp = deepcopy(syear.year)
        
        if eyear.year > sstop_temp: 
            sstop_temp = deepcopy(eyear.year)
            
        ## try to find earliest combination
        if earliest == 5:
            # find all semesters belonging to the earliest year
            earliest = deepcopy(sem)
        else:
            if syear.year < sstart_temp:
                earliest = deepcopy(sem)
            elif syear.year == sstart_temp: #less is better here        
                earliest = list(sem_dict_timing.keys())[min([sem_dict_timing[earliest],sem_dict_timing[sem]])] 
            #else:
                #print('error')
            
        ## try to find latest combination
        if latest == 5:
            # find all semesters belonging to the earliest year
            latest = deepcopy(sem)
        else:
            if eyear.year > sstop_temp:
                latest = deepcopy(sem)
            elif eyear.year == sstop_temp: #less is better here        
                latest = list(sem_dict_timing.keys())[max([sem_dict_timing[earliest],sem_dict_timing[sem]])] 
            #else:
                #print('error')
                
        ## may do some kind of adjustment here later
        #if syear.year != eyear.year: #use start year
        #    eff_year = str(syear.year)
        #else:
        #    eff_year = str(syear.year)
    
    eff_year.append([sstart_temp,earliest])
    end_eff_year.append([sstop_temp,latest])
    sstart.append(sstart_temp) #indexed according to unique orgs
    sstop.append(sstop_temp)

In [561]:
if eff_year[0] == 'init':
    del eff_year[0]

if end_eff_year[0] == 'init':
    del end_eff_year[0]
    
print(eff_year[:3])
print(end_eff_year[:3])
print(len(unique_orgs),len(eff_year))
print(len(sstart),len(sstop))

[[2000, 'Spring'], [2015, 'Fall'], [2005, 'Spring']]
[[2020, 'Fall'], [2020, 'Fall'], [2020, 'Fall']]
327 327
327 327


In [562]:
sem_classes_df = pd.DataFrame(columns=['Id','Avg_Sem_Classes__c','Max_Sem_Classes__c'])
sem_users_df = pd.DataFrame(columns=['Id','Avg_Sem_Users__c','Max_Sem_Users__c'])

sem_classes_df['Id'] = unique_orgs
sem_users_df['Id'] = unique_orgs


In [563]:
## semester calculations (need to match by year and semester name [sem_dict_timing])
# for each org, for each year and semester, find the number of classes and users
for i,ii in enumerate(unique_orgs):
    # build list of times
    sem_start = deepcopy(eff_year[i])
    sem_end = deepcopy(sem_start)
    
    #if sem_start[-1] == 'Spring':
    #    sem_end[-1] = 'Summer'
    #elif sem_start[-1] == 'Summer':
    #    sem_end[-1] = 'Fall'
    #else:
    #    sem_end[0] += 1
    #    sem_end[1] = 'Spring'

    sem_classes = []
    sem_users = []
    
    while sem_end[0]+sem_dict_timing[sem_end[1]] <= end_eff_year[i][0]+sem_dict_timing[end_eff_year[i][1]]:
        this_sem_classes = 0
        this_sem_users = 0
        
        for j in org_ind_dict[ii]:            
            ## ensure valid
            if [datetime.datetime.strptime(sf_df.iloc[j,-1],'%Y-%m-%d').year, sf_df.iloc[j,-2]] == sem_start:
                #print(True)
                this_sem_classes += 1
                this_sem_users += sf_df.iloc[j,1]
            else:
                this_sem_classes += 0
                this_sem_users += 0
            
        sem_classes.append(this_sem_classes)
        sem_users.append(this_sem_users)
        
        
        if sem_start[-1] == 'Spring':
            sem_start[-1] = 'Summer'
        elif sem_start[-1] == 'Summer':
            sem_start[-1] = 'Fall'
        else:
            sem_start[0] += 1
            sem_start[1] = 'Spring'
        
        sem_end = deepcopy(sem_start)
        #if sem_start[-1] == 'Spring':
        #    sem_end[-1] = 'Summer'
        #elif sem_start[-1] == 'Summer':
        #    sem_end[-1] = 'Fall'
        #else:
        #    sem_end[0] += 1
        #    sem_end[1] = 'Spring'
    
    try:
        sem_classes_df.iloc[i,1] = statistics.mean(sem_classes)
        sem_classes_df.iloc[i,2] = max(sem_classes)

        sem_users_df.iloc[i,1] = statistics.mean(sem_users)
        sem_users_df.iloc[i,2] = max(sem_users)
    except:
        print(i)
        print(ii)
        print(sem_classes)
        print(sem_users)
        break

In [564]:
display(sem_classes_df.head(5))
display(sem_classes_df.tail(5))

,Id,Avg_Sem_Classes__c,Max_Sem_Classes__c
0,a0r5w00000V42cCAAR,6.75385,27
1,a0r5w00000V42iyAAB,0.944444,5
2,a0r5w00000V42liAAB,2.78,7
3,a0r5w00000V42c0AAB,0.162791,1
4,a0r5w00000V42ikAAB,1.64151,8


,Id,Avg_Sem_Classes__c,Max_Sem_Classes__c
322,a0r5w00000V42z8AAB,0.333333,1
323,a0r5w00000V43GaAAJ,0.333333,1
324,a0r5w00000V43HQAAZ,0.333333,1
325,a0r5w00000V42c4AAB,0.333333,1
326,a0r5w00000V43DFAAZ,0.333333,1


In [565]:
display(sem_users_df.head(5))
display(sem_users_df.tail(5))

,Id,Avg_Sem_Users__c,Max_Sem_Users__c
0,a0r5w00000V42cCAAR,263.046,1199
1,a0r5w00000V42iyAAB,16.5,82
2,a0r5w00000V42liAAB,160.68,547
3,a0r5w00000V42c0AAB,2.37209,36
4,a0r5w00000V42ikAAB,135.717,1095


,Id,Avg_Sem_Users__c,Max_Sem_Users__c
322,a0r5w00000V42z8AAB,4.66667,14
323,a0r5w00000V43GaAAJ,4.33333,13
324,a0r5w00000V43HQAAZ,7.66667,23
325,a0r5w00000V42c4AAB,3.66667,11
326,a0r5w00000V43DFAAZ,5,15


In [566]:
db_4.send_data(sem_classes_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPJmAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPJmAAP
[Success] Closing job successful. Job ID = 7505w00000THPJmAAP


In [567]:
db_4.check_bulk_status()

{'id': '7505w00000THPJmAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:56:38.000+0000',
 'systemModstamp': '2021-01-13T04:56:39.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [568]:
pprint(db_4.check_bulk_failed_results())

''


In [569]:
db_5.send_data(sem_users_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THPJrAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPJrAAP
[Success] Closing job successful. Job ID = 7505w00000THPJrAAP


In [570]:
db_5.check_bulk_status()

{'id': '7505w00000THPJrAAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:56:40.000+0000',
 'systemModstamp': '2021-01-13T04:56:42.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [571]:
db_5.check_bulk_failed_results()

''

### Point 5 - year-over-year statistics

#### pull weekly usage statistics for all organizations

In [583]:
weekly_yoy = pd.DataFrame(columns=['Id','Weekly_YoY__c'])
weekly_yoy['Id'] = unique_orgs

monthly_yoy = pd.DataFrame(columns=['Id','Monthly_YoY__c'])
monthly_yoy['Id'] = unique_orgs

In [584]:
## sum weekly 
for i,ii in enumerate(unique_orgs):
    # build list of times
    earliest_week = estart[i]
    last_week = lstop[i]
    
    if last_week - earliest_week >= datetime.timedelta(days=365): #need to account for edge cases
        ##### sufficient data to populate
        #### perform weekly average (sum users for that year first, verify numbers before averaging)
        ### determine year length
        year_start = deepcopy(earliest_week)
        year_end = year_start + datetime.timedelta(days=365)
        
        #print(year_start)
        #print(year_end)
        
        temp_weekly_yoy = []
        temp_monthly_yoy = []
        
        donezo = False
        while year_end <= last_week and donezo == False:
            ## calculate total users over said year
            yearly_users = 0
            
            for j in org_ind_dict[ii]: #only sum those instances that fall within the current range
                end_date = datetime.datetime.strptime(sf_df.iloc[j,0],'%Y-%m-%d') 
                start_date = datetime.datetime.strptime(sf_df.iloc[j,-1],'%Y-%m-%d') 
                
                if end_date < year_start or start_date > year_end: #case 1 - ( ) [ ]; case 2- [ ] ( )
                    yearly_users += 0
                else:
                    #case 3 - [ ( ) ], 4 -( [ ] ), 5 - ( [ ) ], 6 - [ ( ] )
                    yearly_users += sf_df.iloc[j,1]
            
            temp_weekly_yoy.append(yearly_users/52)
            temp_monthly_yoy.append(yearly_users/12)
            
            ## update year_end and year_start
            if year_end + datetime.timedelta(days=365) > last_week:
                if year_end == last_week:
                    donezo = True
                else:
                    year_end = deepcopy(last_week)
                    year_start = year_end - datetime.timedelta(days=365)
            else:
                year_start = deepcopy(year_end)
                year_end = year_end + datetime.timedelta(days=365)
            
            #print(year_start)
            #print(year_end)
            #print(last_week)
        
        ## filtering the outputs
        if len(temp_weekly_yoy) >= 5:
            temp_weekly_yoy = temp_weekly_yoy[-5:]
            temp_monthly_yoy = temp_monthly_yoy[-5:]
        #else: do nothing
        
        ## issue with some years having 0 usage by institutions
        ## drop 0s and progress
        while 0 in temp_weekly_yoy:
            del temp_weekly_yoy[temp_weekly_yoy.index(0)]
        
        while 0 in temp_monthly_yoy:
            del temp_monthly_yoy[temp_monthly_yoy.index(0)]
            
        ## calculate percentage and average
        if len(temp_weekly_yoy) > 1:
            weekly_yoy_percents = [(temp_weekly_yoy[ind+1]-val)/val for ind,val in enumerate(temp_weekly_yoy[:-1])]

            weekly_yoy.iloc[i,1] = statistics.mean(weekly_yoy_percents)
        
        if len(temp_monthly_yoy) > 1:
            monthly_yoy_percents = [(temp_monthly_yoy[ind+1]-val)/val for ind,val in enumerate(temp_monthly_yoy[:-1])]

            monthly_yoy.iloc[i,1] = statistics.mean(monthly_yoy_percents)

In [585]:
weekly_yoy['Weekly_YoY__c'] = [i*100 for i in weekly_yoy['Weekly_YoY__c']]
display(weekly_yoy.head(5))
display(weekly_yoy.tail(5))

,Id,Weekly_YoY__c
0,a0r5w00000V42cCAAR,20.152061
1,a0r5w00000V42iyAAB,14.369645
2,a0r5w00000V42liAAB,-6.305802
3,a0r5w00000V42c0AAB,-28.750000
4,a0r5w00000V42ikAAB,25.589226


,Id,Weekly_YoY__c
322,a0r5w00000V42z8AAB,NaN
323,a0r5w00000V43GaAAJ,NaN
324,a0r5w00000V43HQAAZ,NaN
325,a0r5w00000V42c4AAB,NaN
326,a0r5w00000V43DFAAZ,NaN


In [586]:
monthly_yoy['Monthly_YoY__c'] = [i*100 for i in monthly_yoy['Monthly_YoY__c']]
display(monthly_yoy.head(5))
display(monthly_yoy.tail(5))

,Id,Monthly_YoY__c
0,a0r5w00000V42cCAAR,20.152061
1,a0r5w00000V42iyAAB,14.369645
2,a0r5w00000V42liAAB,-6.305802
3,a0r5w00000V42c0AAB,-28.750000
4,a0r5w00000V42ikAAB,25.589226


,Id,Monthly_YoY__c
322,a0r5w00000V42z8AAB,NaN
323,a0r5w00000V43GaAAJ,NaN
324,a0r5w00000V43HQAAZ,NaN
325,a0r5w00000V42c4AAB,NaN
326,a0r5w00000V43DFAAZ,NaN


In [587]:
db_6 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [588]:
db_6.object_id = 'Organization__c'
db_6.external_id = 'Id'

In [589]:
db_6.send_data(weekly_yoy)

[Success] Bulk job creation successful. Job ID = 7505w00000THPK6AAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THPK6AAP
[Success] Closing job successful. Job ID = 7505w00000THPK6AAP


In [592]:
db_6.check_bulk_status()

{'id': '7505w00000THPK6AAP',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T04:58:35.000+0000',
 'systemModstamp': '2021-01-13T04:58:39.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 327,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 848,
 'apiActiveProcessingTime': 697,
 'apexProcessingTime': 0}

In [593]:
pprint(db_6.check_bulk_failed_results())

'"sf__Id","sf__Error",Id,Weekly_YoY__c\n'


In [594]:
db_6.send_data(monthly_yoy)

[Success] Bulk job creation successful. Job ID = 7505w00000THPPVAA5
hello
[Success] CSV upload successful. Job ID = 7505w00000THPPVAA5
[Success] Closing job successful. Job ID = 7505w00000THPPVAA5


In [595]:
db_6.check_bulk_status()

{'id': '7505w00000THPPVAA5',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T05:02:31.000+0000',
 'systemModstamp': '2021-01-13T05:02:32.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [596]:
pprint(db_6.check_bulk_failed_results())

''
